In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import librosa

In [2]:
# Load the data
df = pd.read_csv('D:/musicdata/Data/features_30_sec.csv')


In [3]:
# Define genres
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


In [4]:
# Prepare features and target
X = df.drop(['filename', 'label'], axis=1)
y = df['label']

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [8]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=genres))

Model Accuracy: 0.76

Classification Report:
              precision    recall  f1-score   support

       blues       0.63      0.85      0.72        20
   classical       1.00      1.00      1.00        13
     country       0.76      0.59      0.67        27
       disco       0.83      0.71      0.77        21
      hiphop       0.59      0.87      0.70        15
        jazz       0.95      0.91      0.93        22
       metal       0.86      0.96      0.91        25
         pop       0.75      0.69      0.72        13
      reggae       0.70      0.61      0.65        23
        rock       0.56      0.48      0.51        21

    accuracy                           0.76       200
   macro avg       0.76      0.77      0.76       200
weighted avg       0.76      0.76      0.75       200



In [20]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, duration=30)
    features = {}
    
    # Spectral features
    stft = np.abs(librosa.stft(y))
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    rms = librosa.feature.rms(y=y)
    
    # Harmony and perceptr features
    harmony, perceptr = librosa.effects.hpss(y)
    
    features['length'] = len(y)
    features['chroma_stft_mean'] = np.mean(chroma_stft)
    features['chroma_stft_var'] = np.var(chroma_stft)
    features['rms_mean'] = np.mean(rms)
    features['rms_var'] = np.var(rms)
    features['spectral_centroid_mean'] = np.mean(spec_cent)
    features['spectral_centroid_var'] = np.var(spec_cent)
    features['spectral_bandwidth_mean'] = np.mean(spec_bw)
    features['spectral_bandwidth_var'] = np.var(spec_bw)
    features['rolloff_mean'] = np.mean(rolloff)
    features['rolloff_var'] = np.var(rolloff)
    features['zero_crossing_rate_mean'] = np.mean(zcr)
    features['zero_crossing_rate_var'] = np.var(zcr)
    features['harmony_mean'] = np.mean(harmony)
    features['harmony_var'] = np.var(harmony)
    features['perceptr_mean'] = np.mean(perceptr)
    features['perceptr_var'] = np.var(perceptr)
    features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
    
    for i in range(1, 21):
        features[f'mfcc{i}_mean'] = np.mean(mfcc[i-1])
        features[f'mfcc{i}_var'] = np.var(mfcc[i-1])
    
    return [features[column] for column in X.columns]

In [21]:
def predict_genre(file_path):
    # Extract features from the audio file
    features = extract_features(file_path)
    
    # Scale the features
    features_scaled = scaler.transform([features])
    
    # Predict the genre
    prediction = model.predict(features_scaled)
    probabilities = model.predict_proba(features_scaled)
    
    # Get the predicted genre and its probability
    predicted_genre = prediction[0]
    predicted_prob = max(probabilities[0])
    
    return predicted_genre, predicted_prob


In [22]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\blues\blues.00012.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: blues
Confidence: 0.63


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [23]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\classical\classical.00013.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: classical
Confidence: 0.84


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [24]:
# Example usage
audio_path = r'D:\24h2-solo-200476.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: reggae
Confidence: 0.30


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [25]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\country\country.00059.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: blues
Confidence: 0.23


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [26]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\disco\disco.00066.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: disco
Confidence: 0.85


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [27]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\hiphop\hiphop.00091.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: hiphop
Confidence: 0.50


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [28]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\jazz\jazz.00085.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: jazz
Confidence: 0.33


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [29]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\metal\metal.00077.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: metal
Confidence: 0.83


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [30]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\pop\pop.00063.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: pop
Confidence: 0.76


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [31]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\reggae\reggae.00024.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: reggae
Confidence: 0.57


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [35]:
# Example usage
audio_path = r'D:\musicdata\Data\genres_original\rock\rock.00017.wav'
predicted_genre, confidence = predict_genre(audio_path)
print(f"Predicted Genre: {predicted_genre}")
print(f"Confidence: {confidence:.2f}")

Predicted Genre: metal
Confidence: 0.60


C:\Users\Selvibala\AppData\Local\Temp\ipykernel_11784\1749949187.py:35: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  features['tempo'] = librosa.beat.tempo(y=y, sr=sr)[0]
C:\Users\Selvibala\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
